In [1]:
import sys
import os
path = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(path)

import geopandas as gpd
from api.app.jhm_metric_calcs.jhm_metric import main
import networkx as nx

In [2]:
# read dataset with avg rent price and house locations (geometry)
house_prices = gpd.read_parquet(path + '/api/app/data/houses_price_demo.parquet')
house_prices = house_prices.to_crs(house_prices.estimate_utm_crs())

In [3]:
# read drive nx graph
G_d = nx.read_graphml(path + '/api/app/data/G_drive.graphml')

In [4]:
# define company's potential location
# and convert coordinates crs to local crs
company_location = {"lon": 59.860510, "lat": 30.211518}

In [5]:
class Worker():
    speciality: str
    salary: int

worker1 = Worker()
worker1.speciality = "worker_1"
worker1.salary = 80_000

worker2 = Worker()
worker2.speciality = "worker_2"
worker2.salary = 50_000

In [6]:
worker_and_salary = [worker1, worker2]

In [7]:
res = main(
    gdf_houses=house_prices,
    worker_and_salary=worker_and_salary,
    company_location=company_location,
    graph=G_d,
    cell_size_meters=500)



 K: 1.054 

 D: 0.002


In [8]:
res['gdfs']['worker_1']['grid'].explore(column='Idx_mean')

In [9]:
res['gdfs']['worker_1']['house_points'].explore(column='Idx')